In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd "/content/drive/My Drive/Colab Notebooks/github_repo/w266_final/project_re"

/content/drive/.shortcut-targets-by-id/1kzskPlOj9PLeYfK_UrNXwaloxbg5EzRW/github_repo/w266_final/project_re


In [3]:
%%bash
pip install tqdm boto3 requests regex sentencepiece sacremoses

  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7db379606b18c1ed255986a243822deff29840b8031a41460d5856f3e0d9ef40
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


ERROR: botocore 1.19.18 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [4]:
!pip install transformers
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 1.3MB 13.2MB/s 
     |████████████████████████████████| 890kB 37.9MB/s 
     |████████████████████████████████| 1.1MB 46.7MB/s 
     |████████████████████████████████| 2.9MB 56.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c92876b6a0abf521f884fefff6d163a0c0fce7083aca1a8d5c4583cb35d72d4d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 133kB 15.2MB/s 
     |████████████████████████████████| 133kB 12.7MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 6.8MB 36.7MB/s 
ERROR: botocore 1.19.18 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [5]:
!ls model/

MedClinical.py	model_config.py  __pycache__


In [6]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

import importlib
#from model.MedClinical import Biobert_fc 


from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification, BertForTokenClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from util.tools  import *
from util import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

### Define parameters

In [7]:
DATA_DIR = "data/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'Relation Extraction'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 10
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1


CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

### Create Directories, if doesn't exist

In [8]:
REPORTS_DIR

'reports/Relation Extraction_evaluation_report/'

In [9]:
os.listdir(REPORTS_DIR)

['.ignore', 'report_1']

In [8]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
  REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
  os.makedirs(REPORTS_DIR)
  print("yay")
if not os.path.exists(REPORTS_DIR):
  os.makedirs(REPORTS_DIR)
  REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
  os.makedirs(REPORTS_DIR)
    
# if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
#         raise ValueError("Output directory ({}) already exists and is not empty.".format(OUTPUT_DIR))
if not os.path.exists(OUTPUT_DIR):
  os.makedirs(OUTPUT_DIR)

yay


In [9]:
processor = MultiClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)

In [10]:
print(train_examples[0])

In [11]:
train_examples_len

76318

In [12]:
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
# num_labels, label_list

In [13]:
print(label_list)
print(num_labels)

['Reason-Drug', 'Route-Drug', 'Strength-Drug', 'Frequency-Drug', 'Duration-Drug', 'Form-Drug', 'Dosage-Drug', 'ADE-Drug', 'no relation']
9


In [14]:
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmpy1ica1pj
100%|██████████| 213450/213450 [00:00<00:00, 13848443.71B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpy1ica1pj to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpy1ica1pj
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6

In [16]:
train_examples_for_processing = [(example,  MAX_SEQ_LENGTH, tokenizer) for example in train_examples]

### Convert Inputs to features in parallel threads -- Below 2 cells are commented, as it is needed only one time. We will use cached file from second time onwards

In [19]:
 process_count = cpu_count() - 1
 with Pool(process_count) as p:
     train_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
 with open(DATA_DIR + "train_features.pkl", "wb") as f:
     pickle.dump(train_features, f)

In [17]:
with open(DATA_DIR + "train_features.pkl", "rb") as f:
    train_features = pickle.load(f)

In [18]:
 model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)

#model = Biobert_fc()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /tmp/tmp_q5jzaax
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:pytorch_pretrained_bert.modeling:Weights of BertForSequenceClassification not initialized from pretrained model: ['clas

In [19]:
x = model.to(device)

In [20]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
# param_optimizer

In [21]:

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)

In [22]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [23]:
for f in train_features[0:10]:
  print(f.label_id)

0
1
1
2
1
3
4
2
1
4


In [24]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([int(f.label_id) for f in train_features], dtype=torch.long)



INFO:root:***** Running training *****
INFO:root:  Num examples = 76318
INFO:root:  Batch size = 24
INFO:root:  Num steps = 31790


In [ ]:
# torch.tensor([f.label_id for f in train_features])

In [25]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [27]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

#         logits = model(input_ids, segment_ids, input_mask, labels=None)
        logits = model(input_ids, segment_ids, input_mask)

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1



Epoch:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


0.126775



Epoch:  10%|█         | 1/10 [37:31<5:37:47, 2251.99s/it]

0.034246


0.172484

...

0.000939


0.000664



Epoch:  70%|███████   | 7/10 [4:22:39<1:52:31, 2250.47s/it]

0.176749


0.000610Buffered data was truncated after reaching the output size limit.

In [28]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
torch.save(model, OUTPUT_DIR + 'best_mdel')
# model_to_save.config.to_json_file(output_config_file)
# tokenizer.save_vocabulary(OUTPUT_DIR)

In [30]:
 #from sklearn import metrics
 #metrics.f1_score(df["labels"], logits, average='micro')